# **IEOR 4650  Business Analytics (Fall 2019)**

**Homework 3 [Total points: 60]**

** Due: 11:59 PM, October 16 **

This second homework consists of Python programming practices and the applications to Poisson regression. 

The steps to finish in the homework:

*   Step 1: Make a copy of the file to your LionDrive.
*   Step 2: Work with the copy (an ipynb file).
*    <font color='red'>**Step 3: Rename the copy to your_uni_hw3.ipynb (For example: yd4501_hw3.ipynb).**</font>  
*   Step 4: Submit the copy on CourseWorks.

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)


**All the questions in this homework requires coding**

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import sklearn 
import scipy.special as spsp
from scipy.optimize import minimize
import scipy.stats as spst

#Exercise 1 [Total: 23 points]


##Setting
In this example, we take a closer look at the data we used for Poisson regression. 

This data measures the number of ad exposures each customer got **within 1 day**. 


(1) In the class, we checked the model fitting based on how closely the predicted numbers of customers having different number of exposures match with the samples we actually observe. In this exercise, we will explore how to apply cross-validation to Poisson regression.

(2) We are interested in using this dataset to make meaningful business recommendation.

In [0]:
#Load the dataset
link="https://drive.google.com/open?id=1pjdoOKuczV8Zc6Zw8-yUh6q2tqogN-xt"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Exposure.csv')  
Ad = pd.read_csv('Exposure.csv')
Ad.head(5)



,Cookie_ID,Exposures
0,1,0
1,2,0
2,3,3
3,4,49
4,5,4


## Q1 [10 points]

**[Cross validation]**

In the class, we checked the model fitting based on how closely the predicted numbers of customers having different number of exposures match with the samples we actually observe. However, we were basically doing the model selection and model fitting using the chunk of data. 

For this exercise, let's first separate the data by

* first randomize the data 
* Split our data into 3 sets. 

>> Training: The first 4000 observations

>> Validation: Observation 4001 to 4500

>> Testing: Observation 4501 to 5000.

We then test which model is the best model.

* Simple Poisson
* 2-segment Poisson 
* Zero-inflated Poisson 
* NBD (Poisson-Gamma)

For the model accuracy, let's use the joint log likelihood. Again, the higher the joint log likelihood, the better the model is. 

Based on the result, rank the model performance from the highest to the lowest.





In [0]:
#don't remove this line (otherwise, you lose 2 points)
np.random.seed(30)

#shuffle and split


In [0]:
#Model1


In [0]:
#Model2


In [0]:
#Model3


In [0]:
#Model4


##Q2 [4 points]

Based on the model learned on the training set, report the expected number of exposures for the market using each model you tested in Q1.

## Q3 [9 points]

Assume that we decide to stick to NBD model, we want to use this model to do something meaningful.

(1) Based on the model, how many people do you predict on the testing set to have at most 1 exposure? How does this number match with the actual observation?

(2) How many more days of exposures do we at least need to reduce this number to below 200? [You only need to consider integer days]

(3) Assume a customer had 0 exposures, visualize the posterior distribution of $\lambda$ for this customer.

(4) Assume a customer had 5 exposures, visualize the posterior distribution of $\lambda$ for this customer.



#Exercise 2 [Total: 37  Points]

##Setting

In this exercise, we discuss a model that deals with discrete timing. We will use sports analytics as the background for this study. The data is collected from Kaggle. [link](https://www.kaggle.com/kendallgillies/nflstatistics)

In this dataset, we have the information of all the NFL players who are both currently active and have already retired before 2018. In this question, we are interested in modeling the number of years they were active before they retire. 


In [0]:
#Load the dataset
link="https://drive.google.com/open?id=1voxCbS9xGieGHiEYYCzK61f4BZQWy6SD"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('NFL.csv')  
NFL = pd.read_csv('NFL.csv')
NFL.head(5)



,Age,Birth Place,Birthday,College,Current Status,Current Team,Experience,Height (inches),High School,High School Location,Name,Number,Player Id,Position,Weight (lbs),Years Played
0,NaN,"Grand Rapids , MI",5/23/1921,Notre Dame,Retired,NaN,3 Seasons,71.0,NaN,NaN,"Evans, Fred",NaN,fredevans/2513736,NaN,185.0,1946 - 1948
1,NaN,"Dayton , OH",12/21/1930,Dayton,Retired,NaN,1 Season,70.0,NaN,NaN,"Raiff, Jim",NaN,jimraiff/2523700,NaN,235.0,1954 - 1954
2,56.0,"Temple , TX",9/11/1960,Louisiana Tech,Retired,NaN,1 Season,74.0,NaN,NaN,"Fowler, Bobby",NaN,bobbyfowler/2514295,NaN,230.0,1985 - 1985
3,30.0,"New Orleans , LA",9/30/1986,LSU,Retired,NaN,5 Seasons,73.0,NaN,NaN,"Johnson, Quinn",NaN,quinnjohnson/79593,NaN,255.0,2009 - 2013
4,25.0,"Detroit , MI",3/31/1992,Central Michigan,Active,Pittsburgh Steelers,3rd season,77.0,Clintondale HS,"Clinton Twp.,Macomb Co., MI","Walton, L.T.",96.0,l.t.walton/2552444,DE,305.0,NaN


##Q1 [3 points]
**[Data cleaning]**

We will clean our data using the following steps:
* **Keep only retired players**  
*  Based on the last column, Creat an array that gives the number of active years for each player. For example, if "Year Played" for a player is 1954-1954, the number of active years will be 0, while if  "Year Played" for a player is 1971 to 1974, the number of active years will be 3. You will find  [str.split()](https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/) function useful for this question.

##Q2 [10 points]

**[Application of One-segment model]**

Let's think about what distribution would fit this variable. One assumption we can make is that for each year, there is a probability $\theta$ for this person to retire, probability $1-\theta$ for this person to remain active. We also assume that the years are independent from one another. We are interested in examining the number of years $y$ it takes for retirement to happen. This is modeled by a [Geometric distribution](https://en.wikipedia.org/wiki/Geometric_distribution), which is defined as the distribution that measures the number of failures ($y$) it takes before the first success happens when each trial are indepedently and identically distributed with probability of suceess equals $\theta$.

For a Geometric model, 

$P(y)=(1-\theta)^y \theta$ for  $y=0, 1, 2, ...$,

here $0< \theta< 1$.

$E(y)=\frac{1-\theta}{\theta}$

* Estimate $\hat{\theta}$ for this model  
* What is the expected number of years for a NFL player to be active?
* What is the probability for a player to serve for 4 years?
* Now do a visualization to check the model performance. In this graph

>> Visualize the observed number of players served 0 years, 1 years... using a bar chart 

>> Based on the model, predict the number of players served 0 years, 1 years, ... Visualize in the same graph as well using a scatter plot. 

##Q3 [10 points]
###[Estimating a two-segment model]

The previous model assumes that all the players have the same $\theta$, which might be a flawed assumption. Instead, we might want to model two segment of players: one segment players have high risk of retiring (high $\theta$), while the other segment of players have lower risk of retiring (low $\theta$)

* Use a two-segment model to identify $\theta_1$, $\theta_2$, and $p$
* Give an interpretation of these three estimated parameters in the context of this question.
* For each segment, what is the expected number of years for a player to remain active?
* If a player retired after 3 years of playing, what is the probability for this player to belong to the low risk group?
* Plot the observed v.s. predicted numbers like you did in Q1. Comment on any changes in the model fitting.
 

##Q4 [4 points]
**[Beta Prior]**

Now, we move on to assume that $\theta$ follows a continuous distribution. Again, we would like to make a choice of the distribution we want to use. It is often assumed that $\theta$ follows a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) with $\theta \sim Beta(\alpha, \beta)$, which has the following pdf:

$f(\theta)=\frac{\theta^{\alpha-1}(1-\theta)^{\beta-1}}{B(\alpha,\beta)}= \frac{\theta^{\alpha-1}(1-\theta)^{\beta-1}}{\frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha+\beta)}}$

Here, you can either use spsp.gamma to compute three parts of $B(\alpha,\beta)$ seperately. You can also simply use spsp.beta(alpha,beta) to compute $B(\alpha,\beta)$ directly.

where $\alpha$ and $\beta$ are both positive, $0<\theta<1$. In addition, $E(\theta)=\frac{\alpha}{\alpha+\beta}$

* Reproduce the following lineplot that shows the probability density functions for different $\alpha$ and $\beta$ values.  [The color of the lines does not matter]

You might find [this link](https://pythonforundergradengineers.com/unicode-characters-in-python.html)  useful.
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Beta_distribution_pdf.svg/650px-Beta_distribution_pdf.svg.png
" width="450"/>
</div>
[Source: Wikipedia]



##Q5 [6 points]
**[Geometric-Beta distribution]**

It can be derived that the probability of $y$ of a geometric distribution with a Beta prior (assuming $\theta \sim Beta(\alpha, \beta)$) will give the following PMF.

$P(y)=\frac{B(\alpha+1,y+\beta)}{B(\alpha,\beta)}$

* Estimate $\alpha$ and $\beta$ for this Geometric-Beta distribution.
* What is the probability for a player to serve for 4 years? 
* Plot the observed v.s. predicted numbers like you did in Q1. Comment on any changes in the model fitting.

##Q6 [4 points]
**[Posterior Analysis]**

It can be proved that the posterior of $\theta|y$ still follows a Beta distribution

$\theta|y \sim Beta(\alpha+1,\beta+y)$

* Based on the estimated model, plot the distribution of $\lambda$ given $y=0$. 
* In the same graph, plot the distribution of $\lambda$ when $y=10$.
* Add a legend to the plot.